<a href="https://colab.research.google.com/github/jeffblackadar/image_work/blob/master/Categorized_E_II_and_G_VI_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# From: https://github.com/spmallick/learnopencv/blob/master/KerasCNN-CIFAR/keras-cnn-cifar10.ipynb
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, InputLayer
from keras.utils import to_categorical

In [0]:
# Thanks to
# https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
def getListOfFiles(dirName):
    import os
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            #allFiles = allFiles + getListOfFiles(fullPath)
            print(fullPath)
        else:
            allFiles.append(entry)
            #print(entry)    
    return allFiles 


# Copy images into train, validation and test directories
This only needs to be run once.
20 Images training
10 validation

In [0]:
#Move images
def copy_images(original_dataset_dir, portrait_image_prefix, train_portrait_dir, validation_portrait_dir, test_portrait_dir):
    #run it once time for each class of images
    import os, shutil

    list_of_files = getListOfFiles(original_dataset_dir)
    print(len(list_of_files))
    # adjust the ranges based on how the images should be split.
    # an equal number per class is required.
    #train
    for i in range(0,20):
        src = os.path.join(original_dataset_dir, list_of_files[i])
        if os.path.exists(src):
            dst = os.path.join(train_portrait_dir, list_of_files[i])
            shutil.copyfile(src, dst)

    #validation
    for i in range(20,30):
        src = os.path.join(original_dataset_dir, list_of_files[i])
        if os.path.exists(src):
            dst = os.path.join(validation_portrait_dir, list_of_files[i])
            shutil.copyfile(src, dst)

    #test
    for i in range(30,40):
        src = os.path.join(original_dataset_dir, list_of_files[i])
        if os.path.exists(src):
            dst = os.path.join(test_portrait_dir, list_of_files[i])
            shutil.copyfile(src, dst)


In [0]:
import os
base_dir = '/content/drive/My Drive/coin-image-processor/portraits'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

#make each directory for the types of images
train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

portrait = 'all'

#make each sub-directory
train_portrait_dir = os.path.join(train_dir, portrait)
if not os.path.exists(train_portrait_dir):
    os.mkdir(train_portrait_dir)

validation_portrait_dir = os.path.join(validation_dir, portrait)
if not os.path.exists(validation_portrait_dir):
    os.mkdir(validation_portrait_dir)

test_portrait_dir = os.path.join(test_dir, portrait)
if not os.path.exists(test_portrait_dir):
    os.mkdir(test_portrait_dir)
    
    
copy_images('/content/drive/My Drive/coin-image-processor/photos/elizabeth_young', 'eII', train_portrait_dir, validation_portrait_dir, test_portrait_dir)
copy_images('/content/drive/My Drive/coin-image-processor/photos/george_vi', 'gvi', train_portrait_dir, validation_portrait_dir, test_portrait_dir)

print(train_portrait_dir)
print('total training portrait images:', len(os.listdir(train_portrait_dir)))
print(validation_portrait_dir)
print('total validation portrait images:', len(os.listdir(validation_portrait_dir)))
print(test_portrait_dir)
print('total test portrait images:', len(os.listdir(test_portrait_dir)))


Set up Training, Testing, Validation images.
one_hot_label them:
1,0 = Elizabeth II
0,1 = George VI


In [22]:
#https://blog.francium.tech/build-your-own-image-classifier-with-tensorflow-and-keras-dc147a15e38e
  
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

training_dir = "/content/drive/My Drive/coin-image-processor/portraits/train/all"
test_dir = "/content/drive/My Drive/coin-image-processor/portraits/test/all"
validation_dir = "/content/drive/My Drive/coin-image-processor/portraits/validation/all"

#train_data = '/TensorFlow/ImageData/Vehicles/train'
#test_data = '/TensorFlow/ImageData/Vehicles/test'
train_data = training_dir 
test_data = test_dir
validation_data = validation_dir

def one_hot_label(img):
    label = img.split('.')[0]
    label = label[:3]
    if label == 'eII':
        ohl = np.array([1,0])
    elif label == 'gvi':
        ohl = np.array([0,1])
    return ohl

def train_data_with_label():
    train_images = []
    for i in tqdm(os.listdir(train_data)):
        path = os.path.join(train_data, i)
        print(path)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128))
        #plt.imshow(img)
        print(one_hot_label(i))
        train_images.append([np.array(img), one_hot_label(i)])
        shuffle(train_images)
    return train_images

def test_data_with_label():
    test_images = []
    for i in tqdm(os.listdir(test_data)):
        path = os.path.join(test_data, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)        
        img = cv2.resize(img, (128, 128))
        #plt.imshow(img)
        print(one_hot_label(i))
        test_images.append([np.array(img), one_hot_label(i)])
        #shuffle(test_images)
    return test_images

def validation_data_with_label():
    validation_images = []
    for i in tqdm(os.listdir(validation_data)):
        path = os.path.join(validation_data, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)        
        img = cv2.resize(img, (128, 128))
        #plt.imshow(img)
        print(one_hot_label(i))
        validation_images.append([np.array(img), one_hot_label(i)])
        #shuffle(validation_images)
    return validation_images
  
  
  
training_images = train_data_with_label()
testing_images = test_data_with_label()
validation_images = validation_data_with_label()
tr_img_data = np.array([i[0] for i in training_images]).reshape(-1,128,128,1)
tr_lbl_data = np.array([i[1] for i in training_images])
tst_img_data = np.array([i[0] for i in testing_images]).reshape(-1,128,128,1)
tst_lbl_data = np.array([i[1] for i in testing_images])
val_img_data = np.array([i[0] for i in validation_images]).reshape(-1,128,128,1)
val_lbl_data = np.array([i[1] for i in validation_images])

 22%|██▎       | 9/40 [00:00<00:00, 80.14it/s]

/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2900.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2901.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2903.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2906.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2907.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2910.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2911.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2913.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2914.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2916.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2917.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2920.png
[1 0]
/con

 60%|██████    | 24/40 [00:00<00:00, 75.57it/s]

[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2931.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2933.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2934.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/eII2936.png
[1 0]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3330.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3331.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3340.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3341.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3350.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3351.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3360.png
[0 1]
/content/drive/My Drive/coin-image-processor/portraits/train/all/gvi3361.png
[0 1

  0%|          | 0/20 [00:00<?, ?it/s]

...

[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]



 80%|████████  | 16/20 [00:00<00:00, 77.60it/s]

[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


100%|██████████| 20/20 [00:00<00:00, 74.43it/s]

[0 1]
[0 1]
[0 1]
[0 1]


In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
#         zoom_range=0.2, # randomly zoom into images
#         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        #width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        #height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        #horizontal_flip=True,  # randomly flip images
        #vertical_flip=False)  # randomly flip images


    rescale=1./255,
    shear_range=0.05,
    zoom_range=0.05,
    fill_mode = "nearest",    
    width_shift_range=0.05,
    height_shift_range=0.05,
    rotation_range=20,
    horizontal_flip=False)

In [27]:
def createModel3():
    model = Sequential()
    
    model.add(InputLayer(input_shape=(128,128,1)))
    model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))
    
    model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))
    
    model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=5, padding='same'))
    
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    #number of classes
    # 1,0 E-II
    # 0,1 G-VI
    model.add(Dense(2, activation='softmax'))

    return model
  


model2 = createModel3()

model2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()




batch_size = 20
epochs = 15



# datagen.fit(train_data)
#it doesn't make sense we mix test and validation

# Fit the model on the batches generated by datagen.flow().
history2 = model2.fit_generator(datagen.flow(tr_img_data , tr_lbl_data, batch_size=batch_size),
                              #steps_per_epoch=int(np.ceil(tr_img_data .shape[0] / float(batch_size))),
                              steps_per_epoch=23,
                              epochs=epochs,
                              validation_data=(val_img_data, val_lbl_data),
                              #validation_split=0.3,
                              validation_steps=13,
                              #shuffle=True,
                              workers=4)

model2.evaluate(tst_img_data, tst_lbl_data)


#fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, 
#validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, 
#initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 128, 128, 32)      832       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 26, 26, 50)        40050     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 6, 6, 50)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 6, 6, 80)          100080    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 2, 2, 80)          0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 2, 2, 80)         

[4.029523849487305, 0.75]

In [0]:
plt.figure(figsize=[8,6])
plt.plot(history2.history['loss'],'r',linewidth=3.0)
plt.plot(history2.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

In [0]:
plt.figure(figsize=[8,6])
plt.plot(history2.history['acc'],'r',linewidth=3.0)
plt.plot(history2.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np

#from keras.models import load_model
#model_final = load_model('/content/portrait_model.h5')

#function that reads image, shows it on screen and makes a prediction


def predict_for(img_name):
    #load image from file - VGG16 takes (244,244) input
    #myimg = load_img(img_name, target_size=(300,300))
    myimg = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)        
    myimg = cv2.resize(myimg, (128, 128))
    plt.imshow(myimg)
    plt.show()

    #convert image pixels to array
    myimg = img_to_array(myimg)
    myimg = np.expand_dims(myimg, axis=0)
    # prepare image for the VGG model
    #myimg = preprocess_input(myimg)
    #predict probability for all 1000 classes
    pred=int(model2.predict(myimg)[0][0])
    print(model2.predict(myimg))
    #print('Prediction for %s: %s'%(img_name, class_names[pred]))

predict_for('/content/drive/My Drive/coin-image-processor/portraits/test/george_vi/gvi3490.png')
predict_for('/content/drive/My Drive/coin-image-processor/portraits/test/george_vi/gvi3491.png')
predict_for('/content/drive/My Drive/coin-image-processor/portraits/test/george_vi/gvi3500.png')
predict_for('/content/drive/My Drive/coin-image-processor/portraits/test/george_vi/gvi3511.png')
predict_for('/content/drive/My Drive/coin-image-processor/photos/george_vi/gvi3330.png')
predict_for('/content/drive/My Drive/coin-image-processor/photos/elizabeth_young/eII2903.png')
#predict_for('/content/drive/My Drive/coin-image-processor/gvi_test_full.jfif')
#predict_for('/content/drive/My Drive/coin-image-processor/gvi_test_roughcrop.jpg')
#predict_for('/content/drive/My Drive/coin-image-processor/gvi_test_roundcrop.jpg')
